In [1]:
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
df = pd.read_csv('data/zac2022_train_merged_final_preprocessing.json',keep_default_na=False)

In [3]:
df.head(5)

,Unnamed: 0,id,url,title,text,timestamp,revid
0,0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính __NOEDITSECTION__,2022-05-12T12:46:53Z,68591979
1,1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society Internet Society hay ISOC là ...,2022-01-20T07:59:10Z,67988747
2,2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt Tiếng Việt, cũng gọi là tiếng Việt ...",2022-05-29T03:42:42Z,68660631
3,3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio Ohio (viết tắt là OH, viết tắt cũ là O.) ...",2022-04-17T08:15:22Z,68482118
4,4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California California (phát âm như ""Ca-li-phót...",2022-06-16T15:27:07Z,68738039


In [13]:
from sentence_transformers import SentenceTransformer, util
from pyvi.ViTokenizer import tokenize

In [ ]:
model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

In [30]:
titles = df['title'].apply(tokenize).tolist()
titles

KeyboardInterrupt: 

In [16]:
corpus_embeddings = model.encode(titles,convert_to_tensor=True,show_progress_bar=True,device='cuda')

Batches:   0%|          | 0/39796 [00:00<?, ?it/s]

In [28]:
pickle.dump(corpus_embeddings, open('data/title_embeddings.pkl', 'wb'))

In [17]:
corpus_embeddings.shape

torch.Size([1273469, 768])

In [29]:
q_raw = '''huyện Tuy An'''
q = tokenize(q_raw)
query_embedding = model.encode(q,convert_to_tensor=True,)
print(q)
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=100)[0]

answers =[]
for hit in hits:
    corpus_id = hit['corpus_id']
    doc_score = hit['score']
    title = titles[corpus_id]
    print(f'{title} - {doc_score}')

huyện Tuy_An
Tuy_An - 0.9403116106987
An_Thọ , Tuy_An - 0.8956292867660522
An_Hiệp , Tuy_An - 0.8954504728317261
An_Cư , Tuy_An - 0.8625950217247009
An_Ninh_Đông , Tuy_An - 0.8604766130447388
An_Định , Tuy_An - 0.8470878601074219
An_Ninh_Tây , Tuy_An - 0.8405049443244934
An_Mỹ , Tuy_An - 0.8361472487449646
Phú_Yên ( xã ) - 0.7179363369941711
Phú_Yên - 0.7162452340126038
Nguyễn_Chấn ( Phú_Yên ) - 0.7052743434906006
Danh_sách xã thuộc tỉnh Phú_Yên - 0.6678887605667114
Nguyễn_Thái_Học ( Phú_Yên ) - 0.6575338840484619
Phú_Yên ( định_hướng ) - 0.6423712968826294
Lịch_sử Phú_Yên - 0.636674165725708
Địa_lý Phú_Yên - 0.632871150970459
Tỉnh_ủy Phú_Yên - 0.631587028503418
Sông Bàn_Thạch ( Phú_Yên ) - 0.6271796226501465
Đồng_Xuân ( xã ) - 0.623569667339325
Xuân_Thịnh , Sông_Cầu - 0.6108188033103943
Anrê Phú_Yên - 0.6087225675582886
Ngựa Phú_Yên - 0.6081807017326355
Sông_Cầu ( xã ) - 0.6050340533256531
Nhơn_Thạnh - 0.5994080305099487
Trường Đại_học Phú_Yên - 0.596480131149292
Vĩnh_Hiệp , Vĩnh_Thạn